<a href="https://colab.research.google.com/github/Gus-1003/ISD-invention/blob/main/Eletro_1/Coordinates_of_an_object_in_image_fase1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Bibliotecas e Imports:


In [1]:
import numpy as np
import cv2
from matplotlib import pyplot as plt
 
import os

from google.colab.patches import cv2_imshow

In [2]:
from google.colab import drive
drive.mount('/content/drive/')
%cd /content/drive/MyDrive/

Drive already mounted at /content/drive/; to attempt to forcibly remount, call drive.mount("/content/drive/", force_remount=True).
/content/drive/MyDrive


# Carregando Imagem:

In [3]:
# nome_completo = variável que armazena o endereço da imagem:
nome_endereco_origem = ''

# img_original = variável que armazena a imagem do Drive:
img_original = cv2.imread(nome_endereco_origem)

# Clone para comparação:
clone_img_original = img_original.copy()

# Imagem onde será desenhada os contornos::
img_contours = img_original.copy()

In [ ]:
cv2_imshow(img_original)

## Endereço da pasta e arquivo:

In [ ]:
list_components = nome_endereco_origem.split('/')
list_components[6]

# Pre-processamento:

## Split dos canais:

In [ ]:
# O uso do im_split é no algaritmo do plo do histograma
im_split = cv2.split(img_original)

# Cada letra é um canal:
(blue, green, red) = cv2.split(img_original)

cv2_imshow(cv2.hconcat([blue, green, red]))

## Histograma:

In [ ]:
# numero de bins do histograma
hist_size = 256

# faixa de valores de cada canal
hist_range = (0, 256) #0-255 (8 bits)
accumulate = False

color = ('b','g','r') #h, s, v

for i,col in enumerate(color):

    histr = cv2.calcHist(im_split,[i],None, [hist_size], hist_range, accumulate=accumulate)

    plt.plot(histr, color = col)

    plt.xlim([0,256])

    
plt.show()

## O canal escolhido:

In [8]:
# O canal verde / Green demonstra uma nitidez maior perante os demais
img_base = green

# Segmentação:

## Threshold Global:

Docs do Thresholding: https://docs.opencv.org/4.x/d7/d4d/tutorial_py_thresholding.html

In [ ]:
ret,th1 = cv2.threshold(img_base,125,255,cv2.THRESH_BINARY)
cv2_imshow(th1)

## Aplicação de Filtros:

In [ ]:
th2 = cv2.adaptiveThreshold(img_base,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,11,2)
th3 = cv2.adaptiveThreshold(img_base,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,11,2)
th4 = cv2.adaptiveThreshold(img_base,255,cv2.ADAPTIVE_THRESH_GAUSSIAN_C, cv2.THRESH_BINARY_INV,11,2)

cv2_imshow(cv2.hconcat([th2, th3, th4]))

## Conclusão da segmentação:

Image Filtering: https://docs.opencv.org/3.4/d4/d86/group__imgproc__filter.html#gaeb1e0c1033e3f6b891a25d0511362aeb

In [ ]:
ret,th5 = cv2.threshold(img_base,202,255,cv2.THRESH_BINARY)

# Aplicação do filtro de media em cima de um threshold global de Img_Base:
th6 = cv2.adaptiveThreshold(th5,255,cv2.ADAPTIVE_THRESH_MEAN_C, cv2.THRESH_BINARY_INV,11,2)

cv2_imshow(cv2.hconcat([th5, th6]))

# Contornos:

## FindContours:

In [12]:
# Find contours, draw on image and save
contours, hierarchy = cv2.findContours(th6,cv2.RETR_EXTERNAL,cv2.CHAIN_APPROX_SIMPLE)

## Demonstração do Contorno:

In [13]:
for qtd in range(len(contours)):
  atual = cv2.contourArea(contours[qtd])

print ('Foram encontrados ' + str(len(contours)) + ' objetos com seus contornos')

Foram encontrados 5 objetos com seus contornos


In [ ]:
cv2.drawContours(img_contours, contours, -1, (0,255,0), 2); #unico contorno 0
cv2_imshow(cv2.hconcat([clone_img_original, img_contours]))

## Coleta dos dados - Cortornos:

In [15]:
areas = []
perimeters = []
centroids_x = []
centroids_y = []
aspect_ratio = [] #It is the ratio of width to height of bounding rect of the object.
extent = [] #Extent is the ratio of contour area to bounding rectangle area.
solidity = [] #Solidity is the ratio of contour area to its convex hull area.
equivalent_diameter = [] #Equivalent Diameter is the diameter of the circle whose area is same as the contour area.
width = []
height = []
angle = []
radius = []

for forma in contours:
    M = cv2.moments(forma)
    
    areas.append(M['m00'])
    perimeters.append(cv2.arcLength(forma,True)) #if not convex, False
    
    x,y,w,h = cv2.boundingRect(forma)
    aspect_ratio.append(float(w)/h)
    width.append(w)
    height.append(h)
    rect = cv2.minAreaRect(forma)
    circle = cv2.minEnclosingCircle(forma)
    angle.append(rect[2])
    radius.append(circle[1])
    rect_area = w*h

    if rect_area > 0:
        extent.append(M['m00']/rect_area)
    else: 
        extent.append(0)
    
    hull = cv2.convexHull(forma)
    hull_area = cv2.contourArea(hull)
    if hull_area > 0:
        solidity.append(M['m00']/hull_area)
    else:
        solidity.append(0)
    
    equivalent_diameter = np.sqrt(4*M['m00']/np.pi)
    
    if M['m00'] != 0:
        centroids_x.append(M['m10']/M['m00'])
        centroids_y.append(M['m01']/M['m00'])
    else:
        centroids_x.append(0)
        centroids_y.append(0)

# Processamento de Dados:

## Feature Selecion:

In [16]:
features = {'area': areas, 'perimiter': perimeters, 'aspect_ratio': aspect_ratio, 'eixo x':centroids_x, 'eixo y': centroids_y, 'radius': radius,
            'extent': extent, 'solidity': solidity, 'equivalent_diameter': equivalent_diameter,
            'width': width, 'height': height, 'angle': angle}


## Create Dataframe:

In [17]:
import pandas as pd

df = pd.DataFrame(features)
df

,area,perimiter,aspect_ratio,eixo x,eixo y,radius,extent,solidity,equivalent_diameter,width,height,angle
0,175.5,83.698485,1.000000,1267.725546,707.420703,15.556449,0.331758,0.663516,105.863376,23,23,47.726311
1,60.0,32.000000,0.636364,1276.000000,687.000000,5.831052,0.779221,1.000000,105.863376,7,11,90.000000
2,150.0,50.000000,0.687500,8.000000,402.500000,9.013978,0.852273,1.000000,105.863376,11,16,90.000000
3,9231.5,379.865005,1.056604,745.445901,399.170305,57.404366,0.777586,0.973992,105.863376,112,106,90.000000
4,8802.0,379.622364,0.981308,429.232864,388.817144,55.095718,0.783445,0.975615,105.863376,105,107,86.820168


## Objetos Alvo:

In [18]:
# Dados a patir dos contornos:
for cnt in contours:
   (x,y),radius = cv2.minEnclosingCircle(cnt)
   center = (int(x),int(y))
   radius = int(radius)
   print('Contour: centre {},{}, radius {}'.format(x,y,radius))

Contour: centre 1268.0,708.0, radius 15
Contour: centre 1276.0,687.0, radius 5
Contour: centre 8.0,402.5, radius 9
Contour: centre 743.5,399.0, radius 57
Contour: centre 428.13848876953125,388.9745788574219, radius 55


In [19]:
# Dados filtrados do Dataframe:
coordenadas = df.loc[df['area'] > 8000]

# Reposicionar as linhas:
coordenadas = coordenadas.reset_index(drop=True)

coordenadas.head()

,area,perimiter,aspect_ratio,eixo x,eixo y,radius,extent,solidity,equivalent_diameter,width,height,angle
0,9231.5,379.865005,1.056604,745.445901,399.170305,57.404366,0.777586,0.973992,105.863376,112,106,90.000000
1,8802.0,379.622364,0.981308,429.232864,388.817144,55.095718,0.783445,0.975615,105.863376,105,107,86.820168


# Salvando os Resultados (Image + CSV):

In [20]:
# Endereço da Pasta de saida:
dest = '/content/drive/MyDrive/Exp Contexto/All_Frames/Dados_Image/'

Salvando CSV: https://pt.stackoverflow.com/questions/485418/escrevendo-um-arquivo-csv-no-google-drive-usando-colab

In [23]:
# make a folder by the name of the video file
if not os.path.exists("{}contorno/".format(dest,list_components[6])):
    os.makedirs("{}contorno/".format(dest,list_components[6]))

coordenadas_csv = coordenadas.to_csv('{}contorno/{}.csv'.format(dest,list_components[6]))
cv2.imwrite("{}contorno/{}.jpg".format(dest,list_components[6]), img_contours)

True